In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn import tree

In [2]:
def entropy(y):
    g=[0 for _ in range(2)]
    for i in range(len(y)):
        if(y[i] == 0):
             g[0] += 1
        else:
            g[1] += 1
    ps = np.divide(g,len(y))
    return -np.sum([p * np.log2(p) for p in ps if p > 0])


In [3]:
class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

In [4]:
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=2, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self.grow(X, y)
    
    def predict(self,x):
        
        predicted_label=[self.traverse(x.iloc[i],self.root) for i in range(x.shape[0])]
        return predicted_label

    def grow(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
                or n_labels == 1
                or n_samples < self.min_samples_split):
            leaf_value = self.find_label(y)
            #print('leaf made')
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to gini
        best_feat = self.best_split(X, y, feat_idxs)
        
        # grow the children that result from the split
        left_idxs, right_idxs,best_thresh = self._split(X.iloc[:,best_feat])
        left = self.grow(X.iloc[left_idxs, :], y.iloc[left_idxs], depth+1)
        right = self.grow(X.iloc[right_idxs, :], y.iloc[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)

    def best_split(self, X, y, feat_idxs):
        best_ig = 0
        split_idx= None
        for feat_idx in feat_idxs:
            X_column = X.iloc[:, feat_idx]
            gain =  self.gainratio(X_column,y.to_numpy())
            
            #print(gini[0])
            if  gain > best_ig:
                best_ig = gain
                split_idx = feat_idx
        #print('best ig found at ',split_idx)
        return split_idx 
    
    def infogain(self,X_column,y):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs,threshold = self._split(X_column)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self,data):
        #mean= np.mean(data.iloc[:,node].to_numpy())
        mean= data.mean(axis=0)
        left=[]
        right=[]
        for row in range(data.shape[0]):
            if data.iloc[row]<mean:
                left.append(row)
            else:
                right.append(row)
        #print('split made')
        return left,right,mean
    
    
    def traverse(self, x, node):
        if node.is_leaf_node():
            return node.value
        #print(x[node.feature])
        if x[node.feature] <= node.threshold:
            return self.traverse(x, node.left)
        return self.traverse(x, node.right)

    def find_label(self, y):
        y = list(y)
        s0=0
        s1=0
        for i in range(len(y)):
            if y[i]>0:
                s1+=1
            else:
                s0+=1
        if s0> s1:
            return 0
        else :
            return 1
    def print_tree(self,tree=None,indent=" "):
        if not tree:
            tree = self.root
        if tree.value!=None:
            print(tree.value)
        else:
            print("x "+ df.columns[tree.feature],"<=",tree.threshold)
            print("%sleft:" % (indent),end="")
            self.print_tree(tree.left,indent+indent)
            print("%sright:" % (indent),end="")
            self.print_tree(tree.right,indent+indent)
    
    def gainratio(self,df, label):
    
        gain_split = self.infogain(df, label)
        l = len(df)
        #w = len(df[0])
        mean = df.mean(axis=0)

        #gai = []
        split = []
        left_idxs, right_idxs,threshold = self._split(df)
        observed_t = [[0 for _ in range(2)] for _ in range(2)]
        #expected_t = [[0 for _ in range(2)] for _ in range(2)]
        sum_t = [0 ,0]
        
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        for i in left_idxs:
            if(label[i] == 0):
                    observed_t[0][0] += 1
            else:
                    observed_t[0][1] += 1
        
        for i in right_idxs:
                if(label[i] == 0):
                    observed_t[1][0] += 1
                else:
                    observed_t[1][1] += 1
                    
        sum_t = list(np.sum(observed_t, axis = 1))
         
        split_info = - (sum_t[0]/sum(sum_t)) * math.log(sum_t[0]/sum(sum_t), 2) - (sum_t[1]/sum(sum_t)) * math.log(sum_t[1]/sum(sum_t), 2)
        
        split.append(split_info)
        
        gain = np.divide(gain_split, split)
    
        return gain

In [5]:
df= pd.read_csv('31.csv',header=None)
for row in range(df.shape[0]):
    if df.iloc[row,-1]>0:
        df.iloc[row,-1]=1
    else:
        df.iloc[row,-1]=0
string = 'feature'
df.columns = [string+str(i) for i in range(df.shape[1])]
#print(df.columns)    
max_depth =int(math.log2(df.shape[1]))
#max_depth =4

In [6]:
X= df.iloc[:,:-1]
Y= df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [7]:
classifier= DecisionTree(2,max_depth)
classifier.fit(X_train,y_train)
classifier.print_tree()


x feature7 <= 9.177275891406914
 left:x feature14 <= 0.5447193478003083
  left:x feature14 <= 0.27561422175950034
    left:x feature16 <= 1.1859872611464968
        left:0
        right:0
    right:x feature3 <= 8.753521126760564
        left:0
        right:0
  right:x feature12 <= 0.06512742321934219
    left:x feature7 <= 3.5167785234899327
        left:0
        right:0
    right:0
 right:x feature3 <= 22.612099644128115
  left:x feature5 <= 30.686634460547506
    left:x feature3 <= 14.858734580183048
        left:0
        right:0
    right:x feature1 <= 1.724662162162162
        left:0
        right:0
  right:x feature3 <= 39.29259525521208
    left:x feature1 <= 1.9819458375125376
        left:0
        right:0
    right:x feature4 <= 140.748730964467
        left:0
        right:1


In [8]:
Y_pred= classifier.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, Y_pred))

Accuracy: 0.8556477805846265


In [9]:
clf = DecisionTreeClassifier(criterion='entropy',splitter='best',max_depth=4,random_state=1)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8552869000360881


In [10]:
text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_7 <= 21.50
|   |--- feature_17 <= 11.56
|   |   |--- feature_14 <= 0.27
|   |   |   |--- feature_6 <= 12.50
|   |   |   |   |--- class: 0
|   |   |   |--- feature_6 >  12.50
|   |   |   |   |--- class: 0
|   |   |--- feature_14 >  0.27
|   |   |   |--- feature_7 <= 2.50
|   |   |   |   |--- class: 0
|   |   |   |--- feature_7 >  2.50
|   |   |   |   |--- class: 0
|   |--- feature_17 >  11.56
|   |   |--- feature_14 <= 0.11
|   |   |   |--- feature_17 <= 18.50
|   |   |   |   |--- class: 1
|   |   |   |--- feature_17 >  18.50
|   |   |   |   |--- class: 0
|   |   |--- feature_14 >  0.11
|   |   |   |--- feature_7 <= 7.50
|   |   |   |   |--- class: 0
|   |   |   |--- feature_7 >  7.50
|   |   |   |   |--- class: 0
|--- feature_7 >  21.50
|   |--- feature_4 <= 130.50
|   |   |--- feature_18 <= 2.50
|   |   |   |--- feature_17 <= 144.17
|   |   |   |   |--- class: 0
|   |   |   |--- feature_17 >  144.17
|   |   |   |   |--- class: 1
|   |   |--- feature_18 >  2.50
|   |   |  